# Thinking of an interesting business

An investor dedicated to the creation of emerging businesses and with great knowledge in Entrepreneurship scalability tells me that you are interested in opening a point of sale for fresh vegetables. He asks me to indicate a good place to establish this business and a complete analysis of the place to make a good decision.


<img src=https://cdn.emprender-facil.com/wp-content/uploads/2018/01/58.jpg.webp>

#### 

### Installation of data manipulation packages

In [1]:
pip install beautifulsoup4

     |████████████████████████████████| 112kB 3.7MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install lxml

     |████████████████████████████████| 5.8MB 5.3MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install html5lib

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [5]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

### Collection of basic information required

In [6]:
link = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [7]:
soup = BeautifulSoup(link, 'html5lib')

In [8]:
table1 = soup.find('table',{'class':'wikitable sortable'})
data1 = table1.find_all('td')

In [9]:
counter = 0
table = []
row = []
for details in data1:
    if counter < 3:
        row.append(details.get_text().strip())
        counter+=1
    else:
        table.append(row)
        counter = 0
        row = []
        row.append(details.get_text().strip())
        counter+=1
headers=['Postcode','Borough','Neighbourhood']
df = pd.DataFrame(table, columns=headers)
def cleaning (row):
    if row['Neighbourhood'] == 'Not assigned':
        row['Neighbourhood'] = row['Borough']
    return row
df1 = df.apply(cleaning, axis=1)
df1 = df1[df1['Borough'] != 'Not assigned']

In [10]:
df1.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [11]:
df2 = df1.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x)).reset_index()
df2.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


### Get the geographical coordinates of the neighborhoods

In [12]:
pip install geocoder

     |████████████████████████████████| 102kB 4.8MB/s ta 0:00:011
     |████████████████████████████████| 829kB 28.7MB/s eta 0:00:01
     |████████████████████████████████| 92kB 5.5MB/s  eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
Note: you may need to restart the kernel to use updated packages.


In [13]:
import geocoder # import geocoder

def get_coordinates1 (row):
    # initialize your variable to None
    lat_lng_coords = None
    print(row['Postcode'])
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(row['Postcode']))
        lat_lng_coords = g.latlng
        print('1')

    row['Latitude'] = lat_lng_coords[0]
    row['Longitude'] = lat_lng_coords[1]
    print(row['Latitude'], row['Longitude'])
    return row

coordinates_df = pd.read_csv('Geospatial_Coordinates.csv')
def get_coordinates2 (row):
    temperature_df = coordinates_df[coordinates_df['Postal Code'] == row['Postcode']]
    row['Latitude'] = float(temperature_df['Latitude'])
    row['Longitude'] = float(temperature_df['Longitude'])
    return row
    
df_with_coordinates = df2
df_with_coordinates['Latitude'] = np.nan
df_with_coordinates['Longitude'] = np.nan
df_with_coordinates = df_with_coordinates.apply(get_coordinates2, axis=1)
df_with_coordinates.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Plotting on a map of Toronto the neighborhoods found

In [14]:
import folium

In [15]:
# for the city Toronto, latitude and longtitude are manually extracted in https://www.geodatos.net/coordenadas/canada/ontario/toronto
toronto_latitude = 43.7001114; toronto_longitude = -79.4162979
map_toronto = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 10.7)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_with_coordinates['Latitude'], df_with_coordinates['Longitude'], df_with_coordinates['Borough'], df_with_coordinates['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    

map_toronto

## Use Foursquare to define the best place

In [16]:
CLIENT_ID = 'TMVXKZCOP0WCBAWAQY1RXVCA2BCTOAI5YXVIHJA3DHCUHPSN' # your Foursquare ID
CLIENT_SECRET = 'NTQ5ZJEPBVTDQBW1QV2420XGNFJ0BFBI2N52NJQZKRWJFBI2' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TMVXKZCOP0WCBAWAQY1RXVCA2BCTOAI5YXVIHJA3DHCUHPSN
CLIENT_SECRET:NTQ5ZJEPBVTDQBW1QV2420XGNFJ0BFBI2N52NJQZKRWJFBI2


In [17]:
North_York_data = df_with_coordinates[df_with_coordinates['Borough'] == 'North York'].reset_index(drop=True)
North_York_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M2H,North York,Hillcrest Village,43.803762,-79.363452
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
2,M2K,North York,Bayview Village,43.786947,-79.385975
3,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714
4,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493


In [18]:
North_York_data.shape

(24, 5)

### Already defined the neighborhood, we proceed to find venues

In [19]:
address_North_York = 'North York,Toronto'
latitude_North_York = 43.7668114
longitude_North_York = -79.4162979

In [20]:
map_North_York = folium.Map(location=[latitude_North_York, longitude_North_York], zoom_start=12)

# add markers to map
for lat, lng, label in zip(North_York_data['Latitude'], North_York_data['Longitude'], North_York_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_North_York)  
    
map_North_York

In [21]:
neighborhood_latitude = North_York_data.loc[0, 'Latitude'] # neighbourhood latitude value
neighborhood_longitude = North_York_data.loc[0, 'Longitude'] # neighbourhood longitude value

neighborhood_name = North_York_data.loc[0, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of "{}" are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of "Hillcrest Village" are 43.8037622, -79.3634517.


In [22]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_North_York, longitude_North_York, VERSION, radius, LIMIT)

In [23]:
results = requests.get(url).json()


In [24]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    

In [25]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']  
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,name,categories,lat,lng
0,Toronto Centre for the Arts,Theater,43.766228,-79.414115
1,The Keg,Steakhouse,43.766579,-79.412131
2,Konjiki Ramen,Ramen Restaurant,43.766998,-79.412222
3,Starbucks,Coffee Shop,43.768353,-79.413046
4,Satay Sate,Indonesian Restaurant,43.766690,-79.412100


In [26]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

53 venues were returned by Foursquare.


In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Get venues for each neighborhood in North York


In [28]:
North_York_venues = getNearbyVenues(names=North_York_data['Neighbourhood'],
                                   latitudes=North_York_data['Latitude'],
                                   longitudes=North_York_data['Longitude']
                                  )

Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Bedford Park, Lawrence Manor East
Lawrence Heights, Lawrence Manor
Glencairn
Downsview, North Park, Upwood Park
Humber Summit
Emery, Humberlea


In [29]:
North_York_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hillcrest Village,43.803762,-79.363452,Eagle's Nest Golf Club,43.805455,-79.364186,Golf Course
1,Hillcrest Village,43.803762,-79.363452,New York Fries,43.803664,-79.363905,Fast Food Restaurant
2,Hillcrest Village,43.803762,-79.363452,AY Jackson Pool,43.804515,-79.366138,Pool
3,Hillcrest Village,43.803762,-79.363452,Villa Madina,43.801685,-79.363938,Mediterranean Restaurant
4,Hillcrest Village,43.803762,-79.363452,Duncan Creek Park,43.805539,-79.360695,Dog Run


In [30]:
North_York_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Downsview North, Wilson Heights",22,22,22,22,22,22
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",24,24,24,24,24,24
"CFB Toronto, Downsview East",3,3,3,3,3,3
Don Mills North,6,6,6,6,6,6
Downsview Central,3,3,3,3,3,3
Downsview Northwest,4,4,4,4,4,4
Downsview West,5,5,5,5,5,5
"Downsview, North Park, Upwood Park",4,4,4,4,4,4


In [31]:
# one hot encoding
North_York_onehot = pd.get_dummies(North_York_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
North_York_onehot['Neighborhood'] = North_York_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [North_York_onehot.columns[-1]] + list(North_York_onehot.columns[:-1])
North_York_onehot = North_York_onehot[fixed_columns]

North_York_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Bakery,Bank,Bar,Baseball Field,...,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Women's Store
0,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
North_York_onehot.shape

(255, 106)

In [33]:
North_York_grouped = North_York_onehot.groupby('Neighborhood').mean().reset_index()
North_York_grouped.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Bakery,Bank,Bar,Baseball Field,...,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Women's Store
0,"Bathurst Manor, Downsview North, Wilson Heights",0.0,0.000000,0.000000,0.0,0.0,0.0,0.045455,0.0,0.000000,...,0.0,0.045455,0.0,0.000000,0.0,0.0,0.0,0.045455,0.0,0.0
1,Bayview Village,0.0,0.000000,0.000000,0.0,0.0,0.0,0.250000,0.0,0.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
2,"Bedford Park, Lawrence Manor East",0.0,0.000000,0.041667,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.041667,0.0,0.041667,0.0,0.0,0.0,0.000000,0.0,0.0
3,"CFB Toronto, Downsview East",0.0,0.333333,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
4,Don Mills North,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.166667,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0


In [34]:
North_York_grouped.shape

(23, 106)

In [35]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [36]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = North_York_grouped['Neighborhood']

for ind in np.arange(North_York_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(North_York_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(12)
neighborhoods_venues_sorted.shape

(23, 11)

In [39]:
from sklearn.cluster import KMeans

North_York_data = North_York_data.drop(16)
# set number of clusters
kclusters = 6

North_York_grouped_clustering = North_York_grouped.drop('Neighborhood', 1)


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(North_York_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 3, 3, 1, 3, 5, 3, 3, 1, 2], dtype=int32)

In [40]:
North_York_merged = North_York_data

# add clustering labels
North_York_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
North_York_merged = North_York_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

North_York_merged

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M2H,North York,Hillcrest Village,43.803762,-79.363452,3,Golf Course,Mediterranean Restaurant,Pool,Fast Food Restaurant,Dog Run,Women's Store,Diner,Comfort Food Restaurant,Concert Hall,Construction & Landscaping
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,3,Clothing Store,Coffee Shop,Fast Food Restaurant,Women's Store,Japanese Restaurant,Bus Station,Sporting Goods Shop,Restaurant,Tea Room,Food Court
2,M2K,North York,Bayview Village,43.786947,-79.385975,3,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Discount Store,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop
3,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714,1,Cafeteria,Women's Store,Clothing Store,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
4,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,M2N,North York,Willowdale South,43.770120,-79.408493,5,Ramen Restaurant,Sushi Restaurant,Sandwich Place,Shopping Mall,Pizza Place,Coffee Shop,Restaurant,Café,Hotel,Ice Cream Shop
6,M2P,North York,York Mills West,43.752758,-79.400049,3,Park,Convenience Store,Bank,Bar,Distribution Center,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Cosmetics Shop
7,M2R,North York,Willowdale West,43.782736,-79.442259,3,Discount Store,Home Service,Pharmacy,Pizza Place,Butcher,Coffee Shop,Grocery Store,Fast Food Restaurant,Deli / Bodega,Comfort Food Restaurant
8,M3A,North York,Parkwoods,43.753259,-79.329656,1,Food & Drink Shop,Park,Women's Store,Discount Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop
9,M3B,North York,Don Mills North,43.745906,-79.352188,2,Caribbean Restaurant,Gym / Fitness Center,Café,Baseball Field,Basketball Court,Japanese Restaurant,Women's Store,Discount Store,Construction & Landscaping,Convenience Store


In [41]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location = [latitude_North_York, longitude_North_York], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(North_York_merged['Latitude'], North_York_merged['Longitude'], North_York_merged['Neighbourhood'], North_York_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Review of each of the clusters to make the decision

In [42]:
North_York_merged.loc[North_York_merged['Cluster Labels'] == 0, North_York_merged.columns[[1] + list(range(5, North_York_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,North York,0,Clothing Store,Furniture / Home Store,Women's Store,Miscellaneous Shop,Boutique,Carpet Store,Coffee Shop,Event Space,Vietnamese Restaurant,Gift Shop


In [43]:
North_York_merged.loc[North_York_merged['Cluster Labels'] == 1, North_York_merged.columns[[1] + list(range(5, North_York_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,North York,1,Cafeteria,Women's Store,Clothing Store,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
8,North York,1,Food & Drink Shop,Park,Women's Store,Discount Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop
18,North York,1,Sandwich Place,Italian Restaurant,Restaurant,Coffee Shop,Café,Juice Bar,Comfort Food Restaurant,Fast Food Restaurant,Butcher,Pharmacy
23,North York,1,Baseball Field,Women's Store,Distribution Center,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store


In [44]:
North_York_merged.loc[North_York_merged['Cluster Labels'] == 2, North_York_merged.columns[[1] + list(range(5, North_York_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,North York,2,Caribbean Restaurant,Gym / Fitness Center,Café,Baseball Field,Basketball Court,Japanese Restaurant,Women's Store,Discount Store,Construction & Landscaping,Convenience Store


In [45]:
North_York_merged.loc[North_York_merged['Cluster Labels'] == 3, North_York_merged.columns[[1] + list(range(5, North_York_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,3,Golf Course,Mediterranean Restaurant,Pool,Fast Food Restaurant,Dog Run,Women's Store,Diner,Comfort Food Restaurant,Concert Hall,Construction & Landscaping
1,North York,3,Clothing Store,Coffee Shop,Fast Food Restaurant,Women's Store,Japanese Restaurant,Bus Station,Sporting Goods Shop,Restaurant,Tea Room,Food Court
2,North York,3,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Discount Store,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop
4,North York,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,North York,3,Park,Convenience Store,Bank,Bar,Distribution Center,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Cosmetics Shop
7,North York,3,Discount Store,Home Service,Pharmacy,Pizza Place,Butcher,Coffee Shop,Grocery Store,Fast Food Restaurant,Deli / Bodega,Comfort Food Restaurant
10,North York,3,Asian Restaurant,Gym,Restaurant,Coffee Shop,Beer Store,Sandwich Place,Smoke Shop,Chinese Restaurant,Concert Hall,Japanese Restaurant
11,North York,3,Coffee Shop,Gift Shop,Sandwich Place,Fried Chicken Joint,Frozen Yogurt Shop,Gas Station,Grocery Store,Diner,Deli / Bodega,Ice Cream Shop
12,North York,3,Caribbean Restaurant,Massage Studio,Bar,Falafel Restaurant,Coffee Shop,Women's Store,Discount Store,Concert Hall,Construction & Landscaping,Convenience Store
13,North York,3,Park,Airport,Bus Stop,Discount Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop


In [46]:
North_York_merged.loc[North_York_merged['Cluster Labels'] == 4, North_York_merged.columns[[1] + list(range(5, North_York_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,North York,4,Grocery Store,Park,Shopping Mall,Bank,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop


In [47]:
North_York_merged.loc[North_York_merged['Cluster Labels'] == 5, North_York_merged.columns[[1] + list(range(5, North_York_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,North York,5,Ramen Restaurant,Sushi Restaurant,Sandwich Place,Shopping Mall,Pizza Place,Coffee Shop,Restaurant,Café,Hotel,Ice Cream Shop
